In [170]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import geopandas as gpd
import folium
from shapely.geometry import Point, Polygon
import branca.colormap as cm
from folium.plugins import TimestampedGeoJson

In [2]:
#Reading json file
bikedata=pd.read_json('/Users/alexanderlindell/Documents/Programmering /Python/Sthlm-EbikeVis-/stations2-copy.json')

In [305]:
#Dict with column names 
df={'date':[],'station':[],'occupancy':[],'capacity':[],'long':[],'lat':[]}
df

{'date': [],
 'station': [],
 'occupancy': [],
 'capacity': [],
 'long': [],
 'lat': []}

In [306]:
for index,row in bikedata.iterrows():
    for item in row:
        for cell in item['data']: 
            #print(cell['date'])
            df['occupancy'].append(cell['occupancy'])
            df['capacity'].append(cell['capacity'])
            df['date'].append(cell['date'])
            df['station'].append(item['id'])
            #print(cell['date'],item['coord']['lon'],item['coord']['lat'])
            df['long'].append(item['coord']['lon'])
            df['lat'].append(item['coord']['lat'])


In [308]:
#Setting up dataframe from dict and fix formatting on timestamp. 
data=pd.DataFrame.from_dict(df)
data['date']= pd.to_datetime(df['date'],unit='s')
data['date']=data['date'].dt.strftime('%Y-%m-%dT%H:%M:%S')
data=data.sort_values(by='date')
data

,date,station,occupancy,capacity,long,lat
0,2022-05-19T21:30:00,003b4566-1a1d-4aa5-9b2c-35972a0ac219,4,12,17.993884,59.324599
1,2022-05-19T21:45:01,003b4566-1a1d-4aa5-9b2c-35972a0ac219,4,12,17.993884,59.324599
2,2022-05-19T22:00:02,003b4566-1a1d-4aa5-9b2c-35972a0ac219,4,12,17.993884,59.324599
3,2022-05-19T22:15:02,003b4566-1a1d-4aa5-9b2c-35972a0ac219,4,12,17.993884,59.324599
4,2022-05-19T22:30:03,003b4566-1a1d-4aa5-9b2c-35972a0ac219,6,12,17.993884,59.324599
...,...,...,...,...,...,...
2955,2022-05-20T06:16:08,ffb197fc-9534-42eb-8090-6ff4ccecf7f3,0,16,18.070286,59.341596
2956,2022-05-20T06:31:11,ffb197fc-9534-42eb-8090-6ff4ccecf7f3,0,16,18.070286,59.341596
2957,2022-05-20T06:46:15,ffb197fc-9534-42eb-8090-6ff4ccecf7f3,0,16,18.070286,59.341596
2958,2022-05-20T07:01:18,ffb197fc-9534-42eb-8090-6ff4ccecf7f3,0,16,18.070286,59.341596


In [349]:
#Setting the status colors for occupancy dependent on capacity.
colorbar = cm.StepColormap(colors=['#15B01A','#FFA500','#FF7F00','#FF4500','#8B0000'], vmin=0,vmax=1)
data['color']=list(colorbar(x/y) for x,y in zip(data['occupancy'],data['capacity']))


In [351]:
#Adjusting size of markers. 
def setradius(size):
    if size*0.2 < 4:
        return(4)
    else: 
        return(size*0.2)

In [352]:
features = []    
for _, row in data.iterrows():
    feature = {
        'type': 'Feature',
        'geometry': {
            'type':'Point', 
            'coordinates':[row['long'],row['lat']]
        },
        'properties': {
            'time': row['date'],
            'popup': 1,
            'style': {'color' : ''},
            'icon': 'circle',
            'iconstyle':{
                'fillColor': row['color'],
                'fillOpacity': 1,
                'stroke': 'true',
                'radius': setradius(row['capacity'])
            }
        }
    }
    features.append(feature)

In [356]:
map = folium.Map(location = [59.32760990395156, 18.06760960579676], tiles='Stamen Toner' , zoom_start = 13)
TimestampedGeoJson( features,
                            add_last_point = True,
                           period='PT10M',
                           loop_button=True,
                           time_slider_drag_update=True,
                            ).add_to(map)
map.add_child(colorbar)
map

In [354]:
map.save('SthlmEBikeVis.html')